Most promising result yet with gradient boosted resnet architecture. Unfortunately, the results of previous experiments were not organized properly and I don't have time to run them atm so you will have to take my words on this one XD

In [ ]:
import numpy as np
import keras
import tensorflow as tf
from keras import layers
from keras import models
from keras.datasets import california_housing

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary
from torch.optim.lr_scheduler import StepLR

from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

import sys
import math

import numpy as np
import math

import numpy as np
import keras
import tensorflow as tf
from keras import layers
from keras import models
from keras.datasets import mnist

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary
from torch.optim.lr_scheduler import StepLR

import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

import numpy as np
import pandas as pd
from scipy.special import logit
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import LeaveOneOut, cross_val_predict, StratifiedKFold
from scipy.stats import norm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def calculate_metrics(model, loader):
    model.eval()
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    return accuracy, f1

In [4]:
class CustomDataLoader:
    def __init__(self, features, labels, batch_size=1, validation_size=0.0):

        if validation_size > 0.0:
            train_data, val_data, train_labels, val_labels = train_test_split(
                features, labels, test_size=validation_size, stratify=labels, random_state=42
            )
            
            train_data_tensor = torch.tensor(train_data).float().to(device)
            train_labels_tensor = torch.tensor(train_labels).long().to(device)
            
            val_data_tensor = torch.tensor(val_data).float().to(device)
            val_labels_tensor = torch.tensor(val_labels).long().to(device)
    
            train_dataset = TensorDataset(train_data_tensor, train_labels_tensor)
            val_dataset = TensorDataset(val_data_tensor, val_labels_tensor)

            self.train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
            self.val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        else:
            features_tensor = torch.tensor(features).float().to(device)
            labels_tensor = torch.tensor(labels).long().to(device)

            dataset = TensorDataset(features_tensor, labels_tensor)

            self.train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
            self.val_loader = None

    def get_train_loader(self):
        return self.train_loader
    
    def get_val_loader(self):
        return self.val_loader

In [5]:
def evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, num_features, patience=None):
    best_val_loss, early_val_accuracy, early_train_loss = float('inf'), 0.0, float('inf')
    best_epoch = 0
    patience_counter = 0

    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in custom_train_loader.get_train_loader():        
            optimizer.zero_grad()
            outputs = model(inputs.view(-1, num_features))

            loss = criterion(outputs, labels, model)
            running_loss += loss.item()

            loss.backward()
            optimizer.step()
                        
        model.eval()
        running_val_loss = 0.0

        unregularized_criterion=nn.CrossEntropyLoss()
        with torch.no_grad():
            for inputs, labels in custom_train_loader.get_val_loader():
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs.view(-1, num_features))

                val_loss = unregularized_criterion(outputs, labels)
                running_val_loss += val_loss.item()

            avg_val_loss = running_val_loss / (len(custom_train_loader.get_val_loader()))

            val_accuracy, val_f1 = calculate_metrics(model, custom_train_loader.get_val_loader())

            if (epoch + 1) % 1 == 0:
                print(f'Epoch {epoch+1}, Loss: {running_loss / (len(custom_train_loader.get_train_loader()))}')
                print(f'Validation Loss: {avg_val_loss}')
                print(f'Val Accuracy: {val_accuracy:.4f}, Val F1-score: {val_f1:.4f}')
                print()

            if patience is None:
                continue
            elif avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                early_val_accuracy = val_accuracy
                early_train_loss = running_loss / len(custom_train_loader.get_train_loader())
                best_epoch = epoch
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f'Early stopping triggered. Best validation loss at epoch {best_epoch+1}: {best_val_loss:.4f}')
                print(f'Val Accuracy: {early_val_accuracy:.4f}')
                print(f'Early Train Loss: {early_train_loss}')
                break

In [6]:
class CustomLoss(nn.Module):
    def __init__(self, criterion, l1_lambda, l2_lambda):
        super(CustomLoss, self).__init__()
        self.criterion = criterion
        self.l1_lambda = l1_lambda
        self.l2_lambda = l2_lambda

    def forward(self, outputs, labels, model):
        loss = self.criterion(outputs, labels)
        
        l1_norm = sum(p.abs().sum() for name, p in model.named_parameters() if 'bias' not in name)
        l2_norm = sum(p.pow(2.0).sum() for name, p in model.named_parameters() if 'bias' not in name)
        
        loss += self.l1_lambda * l1_norm + self.l2_lambda * l2_norm
        return loss

In [7]:
data = pd.read_csv('/kaggle/input/forest-cover-type-dataset/covtype.csv')
data = data.dropna()

X = data.drop(
    [
        "Cover_Type",
    ],
    axis=1,
)
y = data["Cover_Type"]

X = pd.get_dummies(X, drop_first=True)
print(X.shape)

(581012, 54)


In [8]:
x_scaler = StandardScaler()
x_scaled = x_scaler.fit_transform(X)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print(y_encoded)

[4 4 1 ... 2 2 2]


In [9]:
custom_train_loader = CustomDataLoader(x_scaled, y_encoded, batch_size=1024, validation_size=0.2)

In [25]:
class CustomLinearLayer(nn.Module):
    def __init__(self, input_size, output_size, init="looks_linear"):
        super(CustomLinearLayer, self).__init__()
        if init == "zero":
            self.linear = nn.Linear(input_size, output_size, bias=True)

            nn.init.zeros_(self.linear.bias)
            nn.init.zeros_(self.linear.weight)
        elif init == "looks_linear":
            if input_size * 2 != output_size:
                print("Output size must be twice that of input size")
                return
                
            self.linear = nn.Linear(input_size, output_size, bias=True)
            nn.init.zeros_(self.linear.bias)

            with torch.no_grad():
                weight = torch.zeros(input_size * 2, input_size)

                for i in range(self.linear.in_features):
                    weight[2 * i, i] = 1
                    weight[2 * i + 1, i] = -1

                self.linear.weight.copy_(weight)
                nn.init.zeros_(self.linear.bias)
                
            """ Example matrix: [
                [1, 0, 0],
                [-1, 0, 0],
                [0, 1, 0],
                [0, -1, 0],
                [0, 0, 1],
                [0, 0, -1]
            ] """

    def forward(self, x):
        return self.linear(x)

In [26]:
class GradientBoostedResnet(nn.Module):        
    def __init__(self, input_size, output_size):
        super(GradientBoostedResnet, self).__init__()
        self.activation = nn.ReLU()
        
        self.first_layers = nn.ModuleList()
        self.last_layers = nn.ModuleList()

        self.starting_layer = CustomLinearLayer(input_size, output_size, init="zero")
        self.layer_size = input_size

    def grow_network(self):
        if len(self.first_layers) == 0:
            for param in self.starting_layer.parameters():
                param.requires_grad = False
        else:
            for param in self.first_layers[-1].parameters():
                param.requires_grad = False

            for param in self.last_layers[-1].parameters():
                param.requires_grad = False
                
        self.first_layers.append(CustomLinearLayer(self.layer_size, self.layer_size * 2, init="looks_linear").to(device))
        self.last_layers.append(CustomLinearLayer(self.layer_size * 2, self.output_size, init="zero").to(device))
        self.layer_size += self.layer_size * 2

    def forward(self, x):
        outputs = [x]
        for layer in self.first_layers:
            concatenated_outputs = torch.cat(outputs, dim=1)
            outputs.append(self.activation(layer(concatenated_outputs)))

        result = self.starting_layer(outputs[0])
        for i in range(len(self.last_layers)):
            result += self.last_layers[i](outputs[i + 1])

        return result

In [27]:
num_epochs = 10
num_features = 54
num_classes = 7

criterion = CustomLoss(criterion=nn.CrossEntropyLoss(), l1_lambda=0.0001 * 0, l2_lambda=0.0001 * 0)
model = GradientBoostedResnet(num_features, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

for name, param in model.named_parameters():
    break
    print(f"Layer: {name}")
    print(f"Shape: {param.shape}")
    print(param)
    
summary(model, input_size=(1, num_features))
evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, num_features, 5)

Epoch 1, Loss: 0.8032033759854439
Validation Loss: 0.685006390538132
Val Accuracy: 0.7159, Val F1-score: 0.7026

Epoch 2, Loss: 0.663845429336447
Validation Loss: 0.6549301874219325
Val Accuracy: 0.7195, Val F1-score: 0.7077

Epoch 3, Loss: 0.6472340231424911
Validation Loss: 0.6463370286581809
Val Accuracy: 0.7215, Val F1-score: 0.7104

Epoch 4, Loss: 0.6414923824140154
Validation Loss: 0.642463708655876
Val Accuracy: 0.7225, Val F1-score: 0.7119

Epoch 5, Loss: 0.6385957904586708
Validation Loss: 0.6402411591588405
Val Accuracy: 0.7230, Val F1-score: 0.7126

Epoch 6, Loss: 0.6368682980537415
Validation Loss: 0.6387839756513897
Val Accuracy: 0.7232, Val F1-score: 0.7131

Epoch 7, Loss: 0.6357230764653714
Validation Loss: 0.6377830223033303
Val Accuracy: 0.7235, Val F1-score: 0.7135

Epoch 8, Loss: 0.635009997479191
Validation Loss: 0.6372149058601313
Val Accuracy: 0.7237, Val F1-score: 0.7137

Epoch 9, Loss: 0.6345531952276103
Validation Loss: 0.6368132264990556
Val Accuracy: 0.7239, 

In [28]:
model.grow_network()
optimizer = optim.Adam(model.parameters(), lr=0.01 / 2)
    
evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, num_features, 5)

Epoch 1, Loss: 0.5589495077401005
Validation Loss: 0.5160402114454069
Val Accuracy: 0.7790, Val F1-score: 0.7753

Epoch 2, Loss: 0.49059425602925505
Validation Loss: 0.47394294043382007
Val Accuracy: 0.7994, Val F1-score: 0.7962

Epoch 3, Loss: 0.4587082071845227
Validation Loss: 0.44848978022734326
Val Accuracy: 0.8122, Val F1-score: 0.8095

Epoch 4, Loss: 0.43784497533863337
Validation Loss: 0.4313589891320781
Val Accuracy: 0.8210, Val F1-score: 0.8187

Epoch 5, Loss: 0.4228144713435404
Validation Loss: 0.4186344196399053
Val Accuracy: 0.8272, Val F1-score: 0.8251

Epoch 6, Loss: 0.41046881931731355
Validation Loss: 0.40851961363825884
Val Accuracy: 0.8320, Val F1-score: 0.8299

Epoch 7, Loss: 0.40130857374962203
Validation Loss: 0.40275643190793825
Val Accuracy: 0.8349, Val F1-score: 0.8326

Epoch 8, Loss: 0.39372779343621844
Validation Loss: 0.39581206467067986
Val Accuracy: 0.8378, Val F1-score: 0.8358

Epoch 9, Loss: 0.3868714071449204
Validation Loss: 0.388724281861071
Val Accur

In [29]:
model.grow_network()
optimizer = optim.Adam(model.parameters(), lr=0.01 / 4)

print(summary(model, input_size=(1, num_features)))
evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, num_features, 5)

Layer (type:depth-idx)                   Output Shape              Param #
GradientBoostedResnet                    [1, 7]                    --
├─ModuleList: 1-3                        --                        (recursive)
│    └─CustomLinearLayer: 2-1            [1, 108]                  --
│    │    └─Linear: 3-1                  [1, 108]                  (5,940)
├─ReLU: 1-2                              [1, 108]                  --
├─ModuleList: 1-3                        --                        (recursive)
│    └─CustomLinearLayer: 2-2            [1, 324]                  --
│    │    └─Linear: 3-2                  [1, 324]                  52,812
├─ReLU: 1-4                              [1, 324]                  --
├─CustomLinearLayer: 1-5                 [1, 7]                    --
│    └─Linear: 2-3                       [1, 7]                    (385)
├─ModuleList: 1-6                        --                        --
│    └─CustomLinearLayer: 2-4            [1, 7]        

In [30]:
model.grow_network()
optimizer = optim.Adam(model.parameters(), lr=0.01 / 8)
    
evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, num_features, 5)

Epoch 1, Loss: 0.21557455880788978
Validation Loss: 0.22951441205907286
Val Accuracy: 0.9089, Val F1-score: 0.9086

Epoch 2, Loss: 0.20991351278211576
Validation Loss: 0.22428394788712785
Val Accuracy: 0.9114, Val F1-score: 0.9111

Epoch 3, Loss: 0.20360031359652592
Validation Loss: 0.21718879004842356
Val Accuracy: 0.9140, Val F1-score: 0.9137

Epoch 4, Loss: 0.196012831141245
Validation Loss: 0.20984994816152672
Val Accuracy: 0.9168, Val F1-score: 0.9166

Epoch 5, Loss: 0.18805074750851955
Validation Loss: 0.20274915990599415
Val Accuracy: 0.9198, Val F1-score: 0.9196

Epoch 6, Loss: 0.18029113438817373
Validation Loss: 0.19611350799861707
Val Accuracy: 0.9216, Val F1-score: 0.9214

Epoch 7, Loss: 0.1731916837689635
Validation Loss: 0.1902189380244205
Val Accuracy: 0.9242, Val F1-score: 0.9240

Epoch 8, Loss: 0.16670487013682395
Validation Loss: 0.185295128508618
Val Accuracy: 0.9263, Val F1-score: 0.9261

Epoch 9, Loss: 0.16096559918530712
Validation Loss: 0.1811738908290863
Val Acc

In [31]:
model.grow_network()
optimizer = optim.Adam(model.parameters(), lr=0.01 / 10)

print(summary(model, input_size=(1, num_features)))
evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, num_features, 5)

Layer (type:depth-idx)                   Output Shape              Param #
GradientBoostedResnet                    [1, 7]                    --
├─ModuleList: 1-7                        --                        (recursive)
│    └─CustomLinearLayer: 2-1            [1, 108]                  --
│    │    └─Linear: 3-1                  [1, 108]                  (5,940)
├─ReLU: 1-2                              [1, 108]                  --
├─ModuleList: 1-7                        --                        (recursive)
│    └─CustomLinearLayer: 2-2            [1, 324]                  --
│    │    └─Linear: 3-2                  [1, 324]                  (52,812)
├─ReLU: 1-4                              [1, 324]                  --
├─ModuleList: 1-7                        --                        (recursive)
│    └─CustomLinearLayer: 2-3            [1, 972]                  --
│    │    └─Linear: 3-3                  [1, 972]                  (473,364)
├─ReLU: 1-6                             

In [32]:
model.grow_network()
optimizer = optim.Adam(model.parameters(), lr=0.01 / 20)

print(summary(model, input_size=(1, num_features)))
evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, num_features, 5)

Layer (type:depth-idx)                   Output Shape              Param #
GradientBoostedResnet                    [1, 7]                    --
├─ModuleList: 1-9                        --                        (recursive)
│    └─CustomLinearLayer: 2-1            [1, 108]                  --
│    │    └─Linear: 3-1                  [1, 108]                  (5,940)
├─ReLU: 1-2                              [1, 108]                  --
├─ModuleList: 1-9                        --                        (recursive)
│    └─CustomLinearLayer: 2-2            [1, 324]                  --
│    │    └─Linear: 3-2                  [1, 324]                  (52,812)
├─ReLU: 1-4                              [1, 324]                  --
├─ModuleList: 1-9                        --                        (recursive)
│    └─CustomLinearLayer: 2-3            [1, 972]                  --
│    │    └─Linear: 3-3                  [1, 972]                  (473,364)
├─ReLU: 1-6                             

In [33]:
evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, num_features, 5)

Epoch 1, Loss: 0.08281082983255912
Validation Loss: 0.1372114044793865
Val Accuracy: 0.9500, Val F1-score: 0.9499

Epoch 2, Loss: 0.08077366985708868
Validation Loss: 0.13677412023146948
Val Accuracy: 0.9502, Val F1-score: 0.9502

Epoch 3, Loss: 0.07869037413058827
Validation Loss: 0.13680850511841608
Val Accuracy: 0.9505, Val F1-score: 0.9505

Epoch 4, Loss: 0.07675887255350924
Validation Loss: 0.13670141994953156
Val Accuracy: 0.9510, Val F1-score: 0.9509

Epoch 5, Loss: 0.07487531753855918
Validation Loss: 0.1367736437864471
Val Accuracy: 0.9512, Val F1-score: 0.9512

Epoch 6, Loss: 0.07307671630828916
Validation Loss: 0.13690020921721793
Val Accuracy: 0.9517, Val F1-score: 0.9517

Epoch 7, Loss: 0.07139141675462282
Validation Loss: 0.13718893503149351
Val Accuracy: 0.9518, Val F1-score: 0.9518

Epoch 8, Loss: 0.06973574450034402
Validation Loss: 0.13702548824642835
Val Accuracy: 0.9522, Val F1-score: 0.9522

Epoch 9, Loss: 0.06813430754951968
Validation Loss: 0.1375094350884881
Val

In [34]:
model.grow_network()
optimizer = optim.Adam(model.parameters(), lr=0.01 / 40)

print(summary(model, input_size=(1, num_features)))
evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, num_features, 5)

Layer (type:depth-idx)                   Output Shape              Param #
GradientBoostedResnet                    [1, 7]                    --
├─ModuleList: 1-11                       --                        (recursive)
│    └─CustomLinearLayer: 2-1            [1, 108]                  --
│    │    └─Linear: 3-1                  [1, 108]                  (5,940)
├─ReLU: 1-2                              [1, 108]                  --
├─ModuleList: 1-11                       --                        (recursive)
│    └─CustomLinearLayer: 2-2            [1, 324]                  --
│    │    └─Linear: 3-2                  [1, 324]                  (52,812)
├─ReLU: 1-4                              [1, 324]                  --
├─ModuleList: 1-11                       --                        (recursive)
│    └─CustomLinearLayer: 2-3            [1, 972]                  --
│    │    └─Linear: 3-3                  [1, 972]                  (473,364)
├─ReLU: 1-6                             